In [57]:
import pandas as pd
import numpy as np
import seaborn as sns
import  matplotlib.pyplot as plt
from sklearn.impute import KNNImputer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,normalize,MinMaxScaler,PowerTransformer,RobustScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from yellowbrick.cluster import KElbowVisualizer
from yellowbrick.cluster import SilhouetteVisualizer
from sklearn.metrics import silhouette_score
import warnings
warnings.filterwarnings("ignore")
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [3]:
df = pd.read_csv("s_prob_3.csv")
df

,purchases,balance,credit_limit,payments,tenure,oneoff_purchases,installments_purchases,cash_advance,prc_full_payment,cash_advance_frequency,cash_advance_trx,purchases_trx,minimum_payments,balance_frequency,oneoff_purchases_frequency,purchases_installments_frequency,churn
0,95.40,40.900749,1000.0,201.802084,12,0.00,95.40,0.000000,0.000000,0.000000,0,2,139.509787,0.818182,0.000000,0.083333,0
1,0.00,3202.467416,7000.0,4103.032597,12,0.00,0.00,6442.945483,0.222222,0.250000,4,0,1072.340217,0.909091,0.000000,0.000000,1
2,773.17,2495.148862,7500.0,622.066742,12,773.17,0.00,0.000000,0.000000,0.000000,0,12,627.284787,1.000000,1.000000,0.000000,0
3,1499.00,1666.670542,7500.0,0.000000,12,1499.00,0.00,205.788017,0.000000,0.083333,1,1,0.000000,0.636364,0.083333,0.000000,0
4,16.00,817.714335,1200.0,678.334763,12,16.00,0.00,0.000000,0.000000,0.000000,0,1,244.791237,1.000000,0.083333,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8945,291.12,28.493517,1000.0,325.594462,6,0.00,291.12,0.000000,0.500000,0.000000,0,6,48.886365,1.000000,0.000000,0.833333,0
8946,300.00,19.183215,1000.0,275.861322,6,0.00,300.00,0.000000,0.000000,0.000000,0,6,0.000000,1.000000,0.000000,0.833333,0
8947,144.40,23.398673,1000.0,81.270775,6,0.00,144.40,0.000000,0.250000,0.000000,0,5,82.418369,0.833333,0.000000,0.666667,0
8948,0.00,13.457564,500.0,52.549959,6,0.00,0.00,36.558778,0.250000,0.166667,2,0,55.755628,0.833333,0.000000,0.000000,0


In [4]:
df["churn"].value_counts()

0    7224
1    1726
Name: churn, dtype: int64

In [7]:
x = df.drop(["churn"],axis = 1)
x

,purchases,balance,credit_limit,payments,tenure,oneoff_purchases,installments_purchases,cash_advance,prc_full_payment,cash_advance_frequency,cash_advance_trx,purchases_trx,minimum_payments,balance_frequency,oneoff_purchases_frequency,purchases_installments_frequency
0,95.40,40.900749,1000.0,201.802084,12,0.00,95.40,0.000000,0.000000,0.000000,0,2,139.509787,0.818182,0.000000,0.083333
1,0.00,3202.467416,7000.0,4103.032597,12,0.00,0.00,6442.945483,0.222222,0.250000,4,0,1072.340217,0.909091,0.000000,0.000000
2,773.17,2495.148862,7500.0,622.066742,12,773.17,0.00,0.000000,0.000000,0.000000,0,12,627.284787,1.000000,1.000000,0.000000
3,1499.00,1666.670542,7500.0,0.000000,12,1499.00,0.00,205.788017,0.000000,0.083333,1,1,0.000000,0.636364,0.083333,0.000000
4,16.00,817.714335,1200.0,678.334763,12,16.00,0.00,0.000000,0.000000,0.000000,0,1,244.791237,1.000000,0.083333,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8945,291.12,28.493517,1000.0,325.594462,6,0.00,291.12,0.000000,0.500000,0.000000,0,6,48.886365,1.000000,0.000000,0.833333
8946,300.00,19.183215,1000.0,275.861322,6,0.00,300.00,0.000000,0.000000,0.000000,0,6,0.000000,1.000000,0.000000,0.833333
8947,144.40,23.398673,1000.0,81.270775,6,0.00,144.40,0.000000,0.250000,0.000000,0,5,82.418369,0.833333,0.000000,0.666667
8948,0.00,13.457564,500.0,52.549959,6,0.00,0.00,36.558778,0.250000,0.166667,2,0,55.755628,0.833333,0.000000,0.000000


In [9]:
y = df["churn"]
y

0       0
1       1
2       0
3       0
4       0
       ..
8945    0
8946    0
8947    0
8948    0
8949    0
Name: churn, Length: 8950, dtype: int64

In [34]:
x_train,x_test,y_train,y_test= train_test_split(x,y,test_size=0.2,random_state=12,stratify=y)

In [35]:
x_train

,purchases,balance,credit_limit,payments,tenure,oneoff_purchases,installments_purchases,cash_advance,prc_full_payment,cash_advance_frequency,cash_advance_trx,purchases_trx,minimum_payments,balance_frequency,oneoff_purchases_frequency,purchases_installments_frequency
946,736.72,51.161976,4500.0,1665.891540,8,736.72,0.00,0.000000,1.000000,0.000000,0,5,54.443091,0.250000,0.500000,0.000000
4348,2659.35,5259.140390,6000.0,2249.612405,9,813.00,1846.35,4851.392018,0.000000,0.333333,34,38,4538.542583,0.888889,0.222222,0.777778
8177,1651.12,266.519480,2000.0,1098.566062,12,909.45,741.67,0.000000,0.181818,0.000000,0,31,164.309823,1.000000,0.500000,0.833333
6697,432.00,823.003516,1500.0,212.326548,12,0.00,432.00,0.000000,0.000000,0.000000,0,12,316.809973,1.000000,0.000000,1.000000
7631,0.00,3377.632916,5000.0,1028.191288,12,0.00,0.00,1241.559904,0.000000,0.083333,2,0,724.601528,1.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2706,0.00,2865.386926,6500.0,1186.905688,12,0.00,0.00,965.180677,0.000000,0.416667,7,0,726.990921,1.000000,0.000000,0.000000
6092,158.64,37.765185,1500.0,178.288853,8,0.00,158.64,0.000000,0.000000,0.000000,0,3,66.521398,0.750000,0.000000,0.250000
7023,1503.26,314.495423,4000.0,1230.180271,12,1192.26,311.00,0.000000,0.000000,0.000000,0,8,177.363751,0.727273,0.166667,0.416667
987,36.00,3421.547800,3500.0,404.993426,12,0.00,36.00,19.042417,0.000000,0.083333,1,1,10849.438090,1.000000,0.000000,0.083333


In [36]:
y_train

946     0
4348    0
8177    0
6697    0
7631    1
       ..
2706    1
6092    0
7023    0
987     0
8310    0
Name: churn, Length: 7160, dtype: int64

In [51]:
desired_minority_samples = int(0.4 *7160)

In [52]:
smote = SMOTE(sampling_strategy={1: desired_minority_samples}, random_state=42)
x_resampled, y_resampled = smote.fit_resample(x_train, y_train)

In [53]:
x_resampled

,purchases,balance,credit_limit,payments,tenure,oneoff_purchases,installments_purchases,cash_advance,prc_full_payment,cash_advance_frequency,cash_advance_trx,purchases_trx,minimum_payments,balance_frequency,oneoff_purchases_frequency,purchases_installments_frequency
0,736.72,51.161976,4500.000000,1665.891540,8,736.72,0.00,0.000000,1.000000,0.000000,0,5,54.443091,0.250000,0.500000,0.000000
1,2659.35,5259.140390,6000.000000,2249.612405,9,813.00,1846.35,4851.392018,0.000000,0.333333,34,38,4538.542583,0.888889,0.222222,0.777778
2,1651.12,266.519480,2000.000000,1098.566062,12,909.45,741.67,0.000000,0.181818,0.000000,0,31,164.309823,1.000000,0.500000,0.833333
3,432.00,823.003516,1500.000000,212.326548,12,0.00,432.00,0.000000,0.000000,0.000000,0,12,316.809973,1.000000,0.000000,1.000000
4,0.00,3377.632916,5000.000000,1028.191288,12,0.00,0.00,1241.559904,0.000000,0.083333,2,0,724.601528,1.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8638,0.00,2872.847252,6478.130464,1166.083088,12,0.00,0.00,954.070485,0.000000,0.427602,7,0,725.892155,1.000000,0.000000,0.000000
8639,0.00,1195.476646,2500.000000,375.546024,12,0.00,0.00,1230.472901,0.000000,0.166667,2,0,356.330716,1.000000,0.000000,0.000000
8640,0.00,7124.394612,7500.000000,2075.771580,12,0.00,0.00,1482.809153,0.000000,0.387346,4,0,1970.875047,1.000000,0.000000,0.000000
8641,0.00,1780.220689,3000.000000,983.240415,11,0.00,0.00,3121.440145,0.000000,0.609190,18,0,460.889841,0.909091,0.000000,0.000000


In [54]:
y_resampled

0       0
1       0
2       0
3       0
4       1
       ..
8638    1
8639    1
8640    1
8641    1
8642    1
Name: churn, Length: 8643, dtype: int64

In [55]:
y_resampled.value_counts()

0    5779
1    2864
Name: churn, dtype: int64

In [58]:
def evaluation(model,x,y):
    y_pred = model.predict(x)
    accuracy = accuracy_score(y, y_pred)
    conf_matrix = confusion_matrix(y, y_pred)
    class_report = classification_report(y,y_pred)
    print(f"Accuracy: {accuracy:.2f}")
    print("Confusion Matrix:")
    print(conf_matrix)
    print("Classification Report:")
    print(class_report)
    

In [60]:
rf =RandomForestClassifier(n_estimators=100, random_state=42)

In [61]:
rf.fit(x_resampled, y_resampled)

RandomForestClassifier(random_state=42)

In [64]:
print("testing data evaluation".center(40,"*"))
evaluation(rf,x_test,y_test)

********testing data evaluation*********
Accuracy: 1.00
Confusion Matrix:
[[1445    0]
 [   0  345]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      1445
           1       1.00      1.00      1.00       345

    accuracy                           1.00      1790
   macro avg       1.00      1.00      1.00      1790
weighted avg       1.00      1.00      1.00      1790



In [65]:
print("trainig data evaluation".center(40,"*"))
evaluation(rf,x_resampled,y_resampled)

********trainig data evaluation*********
Accuracy: 1.00
Confusion Matrix:
[[5779    0]
 [   0 2864]]
Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      5779
           1       1.00      1.00      1.00      2864

    accuracy                           1.00      8643
   macro avg       1.00      1.00      1.00      8643
weighted avg       1.00      1.00      1.00      8643

